In [ ]:
from sqlite3 import connect
import numpy as np
import pandas as pd
from numpy.ma.extras import average
from Application.DbAPI import iso3_dict

In [ ]:
conn = connect("..\\Data\\EnvironmentData.db")
cur = conn.cursor()

In [ ]:
food_df = pd.read_csv("..\\Data\\FoodSecurity\\WFSO_july_2024.csv")
food_df

In [ ]:
%%sql
CREATE TABLE food_insecurity(
    year INT,
    iso3c TEXT,
    country TEXT,
    indicator TEXT,
    value REAL
);

In [ ]:
for index, row in food_df.iterrows():
    cur.execute("INSERT INTO food_insecurity (year, iso3c, country, indicator, value) VALUES (?, ?, ?, ?, ?)",
                (row["year"], row["iso3c"], row["country"], row["indicator_short"], row["value"]))
conn.commit()

In [ ]:
%%sql
SELECT * FROM food_insecurity;

In [ ]:
water_df = pd.read_csv("..\\Data\\WaterSecurity\\water-data.csv")
water_df

In [ ]:
water_df.columns = water_df.iloc[1]
water_df.drop(index=[0, 1], inplace=True)
water_df.reset_index(inplace=True)
water_df.drop(columns=["index"], inplace=True)
water_df

In [ ]:
data = []
years = water_df.columns[4:]

for index, row in water_df.iterrows():
    for year in years:
        data.append({"country": row["Country Name"], 
                     "water_per_capita": row[year], 
                     "iso3c": row["Country Code"], 
                     "year": int(year)})
normalized_df = pd.DataFrame(data).dropna()
normalized_df

In [ ]:
%%sql
CREATE TABLE water_security(
    country TEXT,
    water_per_capita REAL,
    iso3c TEXT,
    year INT
);

In [ ]:
for index, row in normalized_df.iterrows():
    cur.execute("INSERT INTO water_security(country, water_per_capita, iso3c, year) VALUES (?, ?, ?, ?)", 
                (row["country"], row["water_per_capita"], row["iso3c"], row["year"]))
conn.commit()

In [ ]:
%%sql
SELECT * FROM water_security

In [ ]:
energy_pred_df = pd.read_csv("..\\Data\\EnergySecurity\\WEO2024_AnnexA_Free_Dataset_Regions.csv")
energy_pred_df["UNIT"].unique()

In [ ]:
%%sql
CREATE TABLE energy_security_predictions(
    year INT,
    continent TEXT,
    ej_value REAL
);

In [ ]:
conversion_factors = {
    'EJ': 1,                              # Already in EJ
    'Million barrels per day': 0.000611714,  # For crude oil
    'Percentage': None,                   # Cannot convert percentages to EJ
    'Billion cubic metres': 0.0382,       # For natural gas
    'Million tonnes of coal equivalent': 0.0293076,  # Mtce
    'TWh': 0.0036,                        # Terawatt-hours
    'PJ': 0.001,                          # Petajoules
    'Million tonnes': None,               # Requires specific fuel/material for conversion
    'Mt CO2': None                        # Requires emission factor for conversion
}

regions_to_drop = ["World", "European Union", "Eurasia", "Asia Pacific"]

# Function to convert to EJ
def convert_to_ej(value, unit):
    if unit in conversion_factors:
        if unit == 'Mt CO2':
            emission_factor = average([94.6, 73.3, 56.1])
            return (value * 1e6 / emission_factor) * 1e-9  # Convert Mt CO2 to EJ
        elif unit == 'Million tonnes':  # Requires specific material
            energy_content = average([24, 42, 54])
            return (value * energy_content) * 1e-9  # Convert GJ to EJ
        else:  # Handle standard unit conversions
            return value * conversion_factors[unit]
    else:
        print(unit)
        return None  # Unknown unit

energy_pred_df = energy_pred_df[(energy_pred_df["PRODUCT"] == "Total") & 
                                (energy_pred_df["FLOW"] == "Total energy supply") & 
                                (~energy_pred_df["REGION"].isin(iso3_dict)) &
                                (~energy_pred_df["REGION"].isin(regions_to_drop))]
energy_pred_df['VALUE_IN_EJ'] = energy_pred_df.apply(
    lambda r: convert_to_ej(r['VALUE'], r['UNIT']),
    axis=1
)

energy_pred_df = energy_pred_df.groupby(["REGION", "YEAR"], as_index=False)["VALUE_IN_EJ"].sum()
energy_pred_df["REGION"] = energy_pred_df["REGION"].apply(lambda x : "South America" if x == "Central and South America" else x)
energy_pred_df

In [ ]:
for index, row in energy_pred_df.iterrows():
    cur.execute("INSERT INTO energy_security_predictions(year, continent, ej_value) VALUES (?, ?, ?)", (row["YEAR"], row["REGION"], row["VALUE_IN_EJ"]))
conn.commit()

In [ ]:
%%sql
SELECT * FROM energy_security_predictions;

In [ ]:
energy_df = pd.read_csv("..\\Data\\EnergySecurity\\sdgi-7-1-1-access-electricity-2023-national.csv")
energy_df.replace(-9999.0, np.nan, inplace=True)
energy_df

In [ ]:
%%sql
CREATE TABLE energy_security(
    country TEXT,
    percent_no_electricity REAL,
    iso3c TEXT
)

In [ ]:
energy_df = energy_df[pd.notnull(energy_df["NAME_0"]) & pd.notnull(energy_df["SDG711pct"])]
for index, row in energy_df.iterrows():
    cur.execute("INSERT INTO energy_security(country, percent_no_electricity, iso3c) VALUES (?, ?, ?)", (row["NAME_0"], row["SDG711pct"], row["ISO3"]))
conn.commit()

In [ ]:
%%sql
SELECT * FROM energy_security